In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-linear-regression/train.csv
/kaggle/input/random-linear-regression/test.csv


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
np.set_printoptions(precision=2)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

In [3]:
df_test = pd.read_csv('/kaggle/input/random-linear-regression/test.csv')
df_train = pd.read_csv('/kaggle/input/random-linear-regression/train.csv')
df_test.dropna()
df_train.dropna()

,x,y
0,24.0,21.549452
1,50.0,47.464463
2,15.0,17.218656
3,38.0,36.586398
4,87.0,87.288984
...,...,...
695,58.0,58.595006
696,93.0,94.625094
697,82.0,88.603770
698,66.0,63.648685


In [4]:

x_test = df_test['x']
x_test = x_test.values.reshape(-1, 1)
y_test = df_test['y']
y_test = y_test.values.reshape(-1, 1)
x_train = df_train['x']
x_train = x_train.values.reshape(-1, 1)
y_train = df_train['y']
y_train= y_train.values.reshape(-1, 1)
print(x_test.shape,y_test.shape)
print(x_train.shape,y_train.shape)

(300, 1) (300, 1)
(700, 1) (700, 1)


In [5]:
x_cv, x_test, y_cv, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=1)
print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (700, 1)
the shape of the training set (target) is: (700, 1)

the shape of the cross validation set (input) is: (150, 1)
the shape of the cross validation set (target) is: (150, 1)

the shape of the test set (input) is: (150, 1)
the shape of the test set (target) is: (150, 1)


In [6]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
print(f"Computed mean of the training set: {scaler.mean_.squeeze():.2f}")
print(f"Computed standard deviation of the training set: {scaler.scale_.squeeze():.2f}")

Computed mean of the training set: 54.99
Computed standard deviation of the training set: 134.59


In [7]:
mean_value = np.nanmean(y_train)

y_train = np.where(np.isnan(y_train), mean_value, y_train)
linear_model = LinearRegression()
linear_model.fit(x_train_scaled, y_train )

LinearRegression()

In [8]:
yhat = linear_model.predict(x_train_scaled)
print(f"MSE(sklearn): {mean_squared_error(y_train, yhat)/ 2}")
#for loop
error = 0
for i in range(len(yhat)):
    temp = (yhat[i]-y_train[i])**2
    error += temp
mse = error/(2*len(yhat))
print(f"training MSE (for-loop implementation): {mse.squeeze()}")


MSE(sklearn): 403.14636166157965
training MSE (for-loop implementation): 403.14636166157965


In [9]:
x_cv = scaler.transform(x_cv)
print(f"Mean used to scale the CV set: {scaler.mean_.squeeze():.2f}")
print(f"Standard deviation used to scale the CV set: {scaler.scale_.squeeze():.2f}")
yhat = linear_model.predict(x_cv)
print(f"Cross validation MSE: {mean_squared_error(y_cv, yhat) / 2}")


Mean used to scale the CV set: 54.99
Standard deviation used to scale the CV set: 134.59
Cross validation MSE: 388.5695041774969


In [10]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x_train_mapped = poly.fit_transform(x_train)
print(x_train_mapped[:5])


[[  24.  576.]
 [  50. 2500.]
 [  15.  225.]
 [  38. 1444.]
 [  87. 7569.]]


In [11]:
scaler_poly = StandardScaler()
x_train_mapped_scaled = scaler_poly.fit_transform(x_train_mapped)
print(x_train_mapped_scaled[:5])


[[-0.23 -0.04]
 [-0.04 -0.04]
 [-0.3  -0.04]
 [-0.13 -0.04]
 [ 0.24 -0.03]]
